<a href="https://colab.research.google.com/github/jwchen25/Modeling_Lab_Class/blob/main/Simple_Tutorial_ML4Chem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial - Machine Learning in Chemistry

## Table of content

0. Relevant packages
1. Basic handling of molecular data
2. Supervised learning
3. A simple example of regression task in chemistry

# 0. Relevant packages

### RDKit
`RDKit` is an open-source software toolkit for cheminformatics, designed to assist in the analysis and design of small molecules and chemical compounds. It provides a set of libraries and tools for the manipulation and analysis of molecular structures, molecular descriptors, molecular fingerprints, molecular similarity, molecular visualization, and more. The toolkit is widely used in academia, as well as in the pharmaceutical, biotech, and chemical industries for a variety of tasks such as virtual screening, lead optimization, and chemical database management.

### Scikit-learn
Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection, model evaluation, and many other utilities. We will learn to use scikit-learn to do machine learning work. You can also browse the scikit-learn [user guide](https://scikit-learn.org/stable/user_guide.html) and [tutorials](https://scikit-learn.org/stable/tutorial/index.html) for additional details.
### Essential Libraries and Tools 
Scikit-learn depends on two other Python packages, NumPy and SciPy. For plotting and interactive development, you should also install matplotlib, IPython, and the Jupyter Notebook.
- **NumPy** is one of the fundamental packages for scientific computing in Python. In scikit-learn, the NumPy array is the fundamental data structure. Any data you’re using will have to be converted to a NumPy array.
- **SciPy** is a collection of functions for scientific computing in Python. Scikit-learn draws from SciPy’s collection of functions for implementing its algorithms.
- **Matplotlib** is the primary scientific plotting library in Python. It provides functions for making publication-quality visualizations such as line charts, histograms, scatter plots, and so on.
- **Pandas** Python library for data wrangling and analysis. It can ingest from a great variety of file formats and databases, like SQL, Excel files, and comma-separated values (CSV) files.

### XGBoost
XGBoost (eXtreme Gradient Boosting) is an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable. It implements machine learning algorithms under the Gradient Boosting framework. XGBoost provides a parallel tree boosting (also known as GBDT, GBM) that solve many data science problems in a fast and accurate way. You can also browse the [XGBoost Documentation](https://xgboost.readthedocs.io/en/stable/) for additional details.

### DeepChem
DeepChem is a high quality open-source toolchain that democratizes the use of deep-learning in chemistry, biology and materials science. It also provides various tools for dataset loader, splitters, molecular featurization, model construction and hyperparameter tuning. You can also browse the [DeepChem Ducumentation](https://deepchem.readthedocs.io/en/latest/) for additional details.

We will first install the required libraries. We also need `RDKit` library to process and analyze molecules, like calculating molecular descriptors.

In [ ]:
# Install all libraries
! pip install numpy scipy matplotlib scikit-learn pandas rdkit xgboost deepchem mordred pycm

# Download all data
! mkdir data
! wget https://raw.githubusercontent.com/schwallergroup/ai4chem_course/main/notebooks/02%20-%20Supervised%20Learning/data/esol.csv -O data/esol.csv

# 1. Basic handling of molecular data

Let's start looking at a specific molecule to play around with RDKit functionalities, and look at caffeine. 

The name `caffeine` does not contain any structural information on the molecule. Just by having this name a computer does not know how many (heavy) atoms `caffeine` contains and what bonds they form. We need machine-accessible representations. One of the most commonly used ones is `SMILES`.

Firstly, import relevant modules of `RDKit`

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole

## 1.1 SMILES

`SMILES` (Simplified Molecular Input Line Entry System) is a line notation representation of molecular structures. It is a way of representing chemical compounds as strings of characters, which can be easily processed and analyzed by computer algorithms.

Each SMILES string consists of symbols that represent the elements in the molecule, as well as brackets and other characters that describe the bonding between the atoms. For example, the SMILES string for ethanol (C2H5OH) would be `CCO`. In SMILES, each carbon atom is represented by the letter "C", each hydrogen atom by the letter "H", and each oxygen atom by the letter "O". The bonding between the atoms is indicated by the arrangement of the characters in the string.

SMILES is widely used in cheminformatics and computational chemistry, as it provides a compact and standardized way of representing molecular structures in a machine-readable form. This makes it possible to compare and analyze large numbers of chemical compounds, as well as to generate predictions about their properties and behavior.


 Look up the SMILES string of `caffeine` on Wikipedia/PubChem, or use this [link](https://opsin.ch.cam.ac.uk/) to get SMILES from molecule name. 

In [ ]:
caffeine_smiles = '' # fill in the SMILES 

## 1.2 - Creating and visualizing molecules

Let's start with the most basic rdkit action: creating a `Mol` object (or variable, as you prefer). `Mol` objects represent molecules, and can be created from different molecular representations (SMILES, .sdf files, etc.). We will use the basic `MolFromSmiles` function to create a variable `caffeine` representing our caffeine molecule.

In [ ]:
caffeine = Chem.MolFromSmiles(caffeine_smiles)

We can display the value of a variable in the notebook by typing the name and then running the cell. In this case, we can visualize the molecule this way.

In [ ]:
caffeine

Another interesting option is saving the mol object as an image. This way, you can download it or save it in your working directory. We can create an image file using the function `MolToImage` and our mol object as the argument. Check the directory to see the image after running the following code

In [ ]:
from rdkit.Chem import Draw

#Create image file
im = Draw.MolToImage(caffeine)

#Save image as a PNG file in our current directory
im.save('caffeine.png')

But what is actually a Mol object? Nothing simpler than a graph representing the molecule! In this graph, vertices represents the atoms and edges the bonds in the molecule. Therefore, we can retrieve the atoms and bonds from the graph and play with them.

In [ ]:
print("Note: Hydrogen atoms are ignored!")

# Get total number of atoms
n_atoms = caffeine.GetNumAtoms()
print(f'Total number of atoms: {n_atoms}')

# Get total number of bonds
n_bonds = caffeine.GetNumBonds()
print(f'Total number of bonds: {n_bonds}')

We can also get the properties of atoms and bonds.

In [ ]:
IPythonConsole.drawOptions.addAtomIndices = True  # adding indices for atoms
IPythonConsole.drawOptions.addBondIndices = False  # not adding indices for bonds
IPythonConsole.molSize = 300, 300  # the image size of the molecule
caffeine

In [ ]:
atom_symbols = []  # a list of element symbols for all atoms
atom_numbers = []  # a list of the corresponding element numbers of all atoms
for atom in caffeine.GetAtoms():
    atom_symbols.append(atom.GetSymbol())
    atom_numbers.append(atom.GetAtomicNum())

print("The list of element symbols for all atoms:")
print(atom_symbols)
print("\nThe list of the corresponding element numbers of all atoms:")
print(atom_numbers)

In [ ]:
IPythonConsole.drawOptions.addAtomIndices = False  # not adding indices for atoms
IPythonConsole.drawOptions.addBondIndices = True  # adding indices for bonds
IPythonConsole.molSize = 300, 300  # the image size of the molecule
caffeine

In [ ]:
bond_types = []  # a list of bond type for all bonds
for bond in caffeine.GetBonds():
    bond_types.append(str(bond.GetBondType()))

print("The list of bond type for all bonds:")
print(bond_types)

# 2. Supervised learning
Two major types of supervised machine learning problems:
- **Classification**, where the task task is to predict a class label (e.g. what color, what smell, state of aggregation, etc).
- **Regression**, where the task is to predict a real number (e.g. solubility in water, yield, selectivity, etc).

## 2.1 Algorithms
- k-Nearest Neighbors (k-NN)
- Linear Models
- Support Vector Machines (SVM)
- Decision Trees
- Ensembles of Decision Trees
  - Random forests
  - Gradient boosting machines

The package [scikit-learn](https://scikit-learn.org/stable/index.html) lets us create a large number ML models in a very conveinent way.

In [ ]:
# linear regressor
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()

### Exercise: Similarly to the linear regression model above, create a [k-NN regression model](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html) using scikit-learn.

In [ ]:
### YOUR CODE

# import
' your code '

# create knn model
knn_clf = ' your code '

### END

Next time, you can browse the scikit-learn [user guide](https://scikit-learn.org/stable/user_guide.html) to learn about supported algorithms and how to create the model you want.

## 2.2 Model evaluation and data splitting

### Why do we need to split the dataset?

We want models to learn from data so we can use them in the future, but we also need to know how good our models perform when they see new examples, and so we reserve some part of our dataset for `testing`: we keep it to evaluate how the model would do the real world, with data it hasn't seen before.

In addition, typically we implement multiple models and select the best one, but how to assess which one is the best, without revealing the `test set`? Well, take another subset of the data, and this one we call `validation` set. 

In the end, we end up splitting our data into `training` (used for training the models), `validation` (for selecting the models), and `test` (for testing the resulting models). If you have more time, you can read [this article](https://towardsdatascience.com/how-to-split-data-into-three-sets-train-validation-and-test-and-why-e50d22d3e54c) for more details.

### Evaluation metrics
The metrics used to evaluate the ML models are very important. The choice of metrics to use influences how model performance is measured and compared. The main evaluation metrics for regression and classification tasks are illustrated below. If you have more time, you can read this [article](https://blog.knoldus.com/model-evaluation-metrics-for-machine-learning-algorithms/) for more details.

<div align="center">
<img src="https://www.oreilly.com/api/v2/epubs/9781492073048/files/assets/mlbf_0407.png" width="500"/>
</div>

## 2.3 The path to a ML model.
0. Define the task
1. Prepare data & split data
2. Choose the model
3. Train the model
4. Evaluate the model
5. Use the model

# 3. A simple example of regression task in chemistry

**Aqueous solubility is one of the key physical properties of interest to a medicinal or agrochemical chemist**. Solubility affects the uptake/distribution of biologically active compounds in living material and the environment, thus affecting their potential efficacy and marketability. However, solubility determination is a time-consuming experiment, and it is useful to be able to assess solubility in the absence of a physical sample. 

Our goal here will be to build a ML model that can predict **aqueous solubility** of organic molecules.

### We will use the [ESOL dataset](https://pubs.acs.org/doi/10.1021/ci034243x) for this task.
This dataset contains structures and water solubility data for 1128 compounds.

### Load dataset

In [ ]:
import pandas as pd

# load dataset from the CSV file (we have downloaded it in the Section 0)
esol_df = pd.read_csv('data/esol.csv')
esol_df.sample(5)

The original dataset contains 2 columns, the `smiles` column, which encodes the molecular structure, and the `log solubility (mol/L)` columns represents aqueous solubility of molecules, which is the our target for this task.

In [ ]:
# Get NumPy arrays from DataFrame for the input and target
smiles = esol_df['smiles'].values
y = esol_df['log solubility (mol/L)'].values

### Molecular descriptor calculation
We need to convert the SMILES strings of molecules into numerical values that can be used as input to the ML models. Many molecular descriptors can be calculated from SMILES strings using software like `RDKit`, `DeepChem` and [Mordred](https://github.com/mordred-descriptor/mordred).\
For this task we will use [DeepChem featurizers](https://deepchem.readthedocs.io/en/latest/api_reference/featurizers.html) to compute molecular descriptors.

In [ ]:
# Here, we use molecular descriptors from RDKit, like molecular weight, number of valence electrons, maximum and minimum partial charge, etc.
from deepchem.feat import RDKitDescriptors
featurizer = RDKitDescriptors()
features = featurizer.featurize(smiles)
print(f"Number of generated molecular descriptors: {features.shape[1]}")

# Drop the features containing invalid values
import numpy as np
features = features[:, ~np.isnan(features).any(axis=0)]
print(f"Number of molecular descriptors without invalid values: {features.shape[1]}")

**Exercise**: You can try to use [MACCSKeysFingerprint](https://deepchem.readthedocs.io/en/latest/api_reference/featurizers.html#maccskeysfingerprint) in [DeepChem featurizers](https://deepchem.readthedocs.io/en/latest/api_reference/featurizers.html) to compute molecular descriptors.

In [ ]:
### YOUR CODE

# import the featurizer
' your code '

# create featurizer
mf_featurizer = ' your code '

# compute molecular descriptors
mf_features = ' your code '

### Feature Selection

Feature selection is a crucial step in machine learning that involves selecting the most relevant features (or variables) from a dataset. This process helps to improve the accuracy and efficiency of a model by reducing the amount of noise and redundancy in the data. Essentially, feature selection allows us to focus on the most important information in the dataset, while ignoring the irrelevant or redundant information.

Many classes and functions in the [sklearn.feature_selection](https://scikit-learn.org/stable/modules/feature_selection.html) module can be used for feature selection.\
Here we show a simple feature selection using `VarianceThreshold` in `scikt-learn`.

In [ ]:
# Here, we removed all zero-variance features, i.e. features that have the same value in all samples.
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=0.0)
features = selector.fit_transform(features)
print(f"Number of molecular descriptors after removing zero-variance features: {features.shape[1]}")

### Dataset split

In [ ]:
from sklearn.model_selection import train_test_split
X = features
# training data size : test data size = 0.8 : 0.2
# fixed seed using the random_state parameter, so it always has the same split.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=0)

### Data preprocessing

As it turns out, different features have different scales and distributions.\
Think of molecular weight, which goes from **0 to around 1000 u.a.m. for small organic molecules**, and electrochemical potential, [typically between -3 and 3](https://par.nsf.gov/servlets/purl/10016877). These differences have a huge impact on ML models, which is the reason why we will normalize all the features.

Many classes and functions in the [sklearn.preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html) module can be used for preprocessing data. Here we show a simple Min-Max normalization of features using `MinMaxScaler` in `scikt-learn`.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

# save original X
X_train_ori = X_train
X_test_ori = X_test
# transform data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Q: Is there a difference between doing data preprocessing before and after data split? Which one is better and why?

### Create models

In [ ]:
# random forest regressor, and the default criterion is mean squared error (MSE)
from sklearn.ensemble import RandomForestRegressor
ranf_reg = RandomForestRegressor(n_estimators=10, random_state=0)  # using 10 trees and seed=0

# XGBoost regressor
from xgboost import XGBRegressor
xgb_reg = XGBRegressor(n_estimators=10, random_state=0)  # using 10 trees and seed=0

### Train and evaluate the models
- Mean Squared Error: $MSE$ = $\frac{1}{n} \Sigma_{i=1}^n({y}-\hat{y})^2$
- Root Mean Squared Error: $RMSE$ = $\sqrt{MSE}$ = $\sqrt{\frac{1}{n} \Sigma_{i=1}^n({y}-\hat{y})^2}$

We choose `RMSE` as the evaluation metric for this task.

In [ ]:
from sklearn.metrics import mean_squared_error

def train_test_model(model, X_train, y_train, X_test, y_test):
    """
    Function that trains a model, and tests it.
    Inputs: sklearn model, train_data, test_data
    """
    # Train model
    model.fit(X_train, y_train)
    
    # Calculate RMSE on training
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    model_train_mse = mean_squared_error(y_train, y_pred_train)
    model_test_mse = mean_squared_error(y_test, y_pred_test)
    model_train_rmse = model_train_mse ** 0.5
    model_test_rmse = model_test_mse ** 0.5
    print(f"RMSE on train set: {model_train_rmse:.3f}, and test set: {model_test_rmse:.3f}.\n")


# Train and test the random forest model
print("Evaluating Random Forest Model.")
train_test_model(ranf_reg, X_train, y_train, X_test, y_test)

# Train and test XGBoost model
print("Evaluating XGBoost model.")
train_test_model(xgb_reg, X_train, y_train, X_test, y_test)

### Q: Compare the results. Which model is better in this case?

**Exercise**: Try to train a [SVM model](https://scikit-learn.org/stable/modules/svm.html#regression) and evaluate it. You can create a [SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR) model using default parameters.

In [ ]:
### YOUR CODE

# import
' your code '

# create a model
svm_reg = ' your code '

# train and evaluate the model
' your code '

### END

Perfect! Now you have mastered training and evaluating the model you want.

## Cross-validation and hyperparamter optimization

Our last topic in this section is hyperparameter optimization.

As you've seen, models need some parameters as input, and we need to decide which are the best parameters (e.g. `n_estimators` in tree-based models). Many classes and functions in the [sklearn.model_selection](https://scikit-learn.org/stable/model_selection.html) module can be used for cross validation and hyperparamter optimization.\
Here, we use cross validation and grid search methods to optimize the paramters of random forest model. In cross-validation, the original training data is further split into training and validation sets.

The class [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) does all the work for us, combining cross-validation with grid search, so we can very easily optimize the hyperparameters.

<div align="center">
<img src="https://scikit-learn.org/stable/_images/grid_search_workflow.png" width="400"/>
</div>


<font size=5 color="red">
Note that when you call grid_search.fit(), you only pass the training data as argument.
</font>
Why?

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10, 20, 30]
}

# use 5-folds cross validation during grid searching
grid_search = GridSearchCV(
    RandomForestRegressor(random_state=0),
    param_grid,
    cv=5
)
grid_search.fit(X_train, y_train)

# re-train a model using best hyperparameters
rf_gs = RandomForestRegressor(**grid_search.best_params_, random_state=0)

print('Best paramters: ', grid_search.best_params_)
print('Random forests performance after hyperparamter optimization:')
train_test_model(rf_gs, X_train, y_train, X_test, y_test)